### Loading dataset

In [6]:
import pandas as pd

In [7]:
path = '/content/reformatted_stock_tweets.csv'

data = pd.read_csv(path)

data.head()

,Unnamed: 0,Date,Tweet,Stock Name,Company Name,Close
0,0,9/29/2022,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc.",268.209992
1,1,9/29/2022,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc.",268.209992
2,2,9/29/2022,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc.",268.209992
3,3,9/29/2022,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc.",268.209992
4,4,9/29/2022,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc.",268.209992


### Pipeline for sentiment analysis


In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

In [ ]:
# sentiment-analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

# Apply sentiment analysis to the 'Tweet' column and create a 'Predictions' column
data['Predictions'] = data['Tweet'].apply(lambda tweet: sentiment_pipeline(tweet)[0]['label'])
data.head()
# Save the results
# data.to_csv('/content/drive/MyDrive/praksa_manu/distilroberta_model_training/dataset_with_predictions.csv', index=False)


It appears a problem here. The dataset is too big and when i try to predict the data my RAM memory is filling up and the session is aborting. As a solution to this I will try to read the dataset partially and predict the data like that.

### Predicting the data partially


In [ ]:
import numpy as np

In [ ]:
chunk_size = 1

data_chunks = np.array_split(data, len(data) // chunk_size + 1)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
chunk_1 = data_chunks[0]
print(chunk_1.head())

                                               Tweet     Close     Month  \
0  Mainstream media has done an amazing job at br...  0.377748  0.727273   
1  Tesla delivery estimates are at around 364k fr...  0.377748  0.727273   
2  3/ Even if I include 63.0M unvested RSUs as of...  0.377748  0.727273   
3  @RealDanODowd @WholeMarsBlog @Tesla Hahaha why...  0.377748  0.727273   
4  @RealDanODowd @Tesla Stop trying to kill kids,...  0.377748  0.727273   

   Year  Month_sin     Month_cos  Year_sin  Year_cos  \
0   1.0       -1.0 -1.836970e-16  0.955237  0.295841   
1   1.0       -1.0 -1.836970e-16  0.955237  0.295841   
2   1.0       -1.0 -1.836970e-16  0.955237  0.295841   
3   1.0       -1.0 -1.836970e-16  0.955237  0.295841   
4   1.0       -1.0 -1.836970e-16  0.955237  0.295841   

                                           Token_IDs  \
0  [0, 31359, 8656, 433, 34, 626, 41, 2770, 633, ...   
1  [0, 39971, 2996, 2785, 32, 23, 198, 38930, 330...   
2  [0, 246, 73, 1648, 114, 38, 680, 55

In [ ]:
all_predictions = []

for chunk in data_chunks:

    input_ids = torch.tensor(chunk['Token_IDs'].tolist())
    attention_mask = torch.tensor(chunk['Attention_mask'].tolist())


    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    predictions = torch.argmax(outputs.logits, axis=-1)


    all_predictions.extend(predictions.tolist())

data['Predictions'] = all_predictions
data.to_csv('predictions_output.csv', index=False)